In [ ]:
import matplotlib.pyplot as plt
from ai2thor.controller import Controller
import ai2thor.robot_controller
import datetime
import os
import numpy as np
import torch
import cv2
import sys
sys.path.append('../')
from utils.hacky_viz_utils import depth_to_rgb
import time
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

# controller = ai2thor.robot_controller.Controller(host="stretch3", port=9000, width=1280, height=720)
env_args = dict(
    width = 400,
    height = 300,
#     fieldOfView = 58,
#     fieldOfView = 42,
    fieldOfView = 63.45,
)
controller = ai2thor.controller.Controller(renderDepthImage=True, renderInstanceSegmentation=True, branch='legacy_asset_fixes', agentMode='locobot', **env_args)
controller._build.url

In [ ]:
controller.reset('FloorPlan_RoboTHOR_Real')

In [ ]:
target_loc = [o for o in controller.last_event.metadata['objects'] if o['objectType'] == 'Sofa'][0]['position']
target_loc = torch.Tensor([target_loc[k] for k in ['x','y','z']])
controller.step('GetReachablePositions')
reachable_position = controller.last_event.metadata['actionReturn']
reachable_position = torch.Tensor([[obj[k] for k in ['x','y','z']] for obj in reachable_position])
values, indices = torch.topk(torch.norm(reachable_position - target_loc, dim=-1), 20, largest=False)
proximity_agent_locations = torch.Tensor([reachable_position[i].numpy() for i in indices])
pencil_loc = [o for o in controller.last_event.metadata['objects'] if o['objectType'] == 'Pencil'][0]['position']
pencil_loc = torch.Tensor([pencil_loc[k] for k in ['x','y','z']])
values, indices = torch.topk(torch.norm(proximity_agent_locations - pencil_loc, dim=-1), 5, largest=False)
indices = [indices[2]]
for ind in indices:
    initial_pose = proximity_agent_locations[ind].numpy()
    event1 = controller.step(
        dict(
            action="TeleportFull",
            x=initial_pose[0],
            y=initial_pose[1],
            z=initial_pose[2],
            rotation=dict(x=0, y=90, z=0),
            horizon=0,
        )
    )
#     plt.imshow(controller.last_event.frame)
#     plt.show()
    plt.imsave('simul.png', controller.last_event.frame)
    time.sleep(2)
    

In [ ]:
target_loc = [o for o in controller.last_event.metadata['objects'] if o['objectType'] == 'Sofa'][0]['position']
target_loc = torch.Tensor([target_loc[k] for k in ['x','y','z']])
controller.step('GetReachablePositions')
reachable_position = controller.last_event.metadata['actionReturn']
reachable_position = torch.Tensor([[obj[k] for k in ['x','y','z']] for obj in reachable_position])
values, indices = torch.topk(torch.norm(reachable_position - target_loc, dim=-1), 20, largest=False)
# proximity_agent_locations = torch.Tensor([reachable_position[i].numpy() for i in indices])
# pencil_loc = [o for o in controller.last_event.metadata['objects'] if o['objectType'] == 'Pencil'][0]['position']
# pencil_loc = torch.Tensor([pencil_loc[k] for k in ['x','y','z']])
# values, indices = torch.topk(torch.norm(proximity_agent_locations - pencil_loc, dim=-1), 5, largest=False)
# indices = [indices[2]]
indices = [indices[0]]
for ind in indices:
    initial_pose = reachable_position[ind].numpy()
    event1 = controller.step(
        dict(
            action="TeleportFull",
            x=initial_pose[0],
            y=initial_pose[1],
            z=initial_pose[2],
            rotation=dict(x=0, y=90, z=0),
            horizon=0,
        )
    )
#     plt.imshow(controller.last_event.frame)
#     plt.show()
    plt.imsave('simul.png', controller.last_event.frame)
    time.sleep(2)
    

In [ ]:
host = 'loco4'
# real_controller_args = {"host":host+".corp.ai2", "port":9000, "width":1280, "height":720}
host = '172.16.121.184'
real_controller_args = {"host":host, "port":9000, "width":1280, "height":720}
controller = ai2thor.robot_controller.Controller(real_controller_args)
# controller.step({"action": "Initialize"})
controller.step({"action": "Pass"})